In [85]:
import os
from dotenv import load_dotenv
load_dotenv()

from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import BaseMessage

class State(TypedDict):
    prompt: str
    output: str
    
class StateWithMessages(TypedDict):
    messages: List[BaseMessage]
    
class StateUserAI(TypedDict):
    user_message: List[str]
    ai_message: List[str]

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18", 
                 api_key=os.getenv("OPENAI_API_KEY"),
                 max_tokens=500)


In [86]:
def process_prompt(state: State) -> State:
    prompt = state.get('prompt', '')
    # Process the prompt (e.g., convert to uppercase)
    response = prompt.upper()
    # Update the state with the response
    state['output'] = response
    return state

In [87]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

def process_prompt_llm(state: StateWithMessages) -> StateWithMessages:
    system_prompt = SystemMessage(content="You are an assistant to break down a user's query into sub questions. Examine the query and break it down into relevant and appropriate sub questions that can be answered.")
    messages = state.get('messages', [])
    
    print('State:', state)
    user_input = messages[-1].content if messages else ''
    
    messages.append(system_prompt)
    messages.append(HumanMessage(content=user_input))
    
    response = llm.invoke(messages)
    
    messages.append(AIMessage(content=response.content))
    
    state['messages'] = messages
    return state


In [88]:
def process_prompt_user_llm(state: StateUserAI) -> StateUserAI:
    system_prompt = SystemMessage(content="You are an assistant to break down a user's query into sub questions. Examine the query and break it down into relevant and appropriate sub questions that can be answered.")
    user_messages = state.get('user_message', [])
    ai_messages = state.get('ai_message', [])
    
    print('State:', state)
    user_input = user_messages[-1].content if user_messages else ''
    messages = []
    messages.append(system_prompt)
    
    messages.append(HumanMessage(content=user_input))
    print('Messages:', messages)
    
    response = llm.invoke(messages)
    
    messages.append(AIMessage(content=response.content))
    
    state['user_message'] = user_messages
    state['ai_message'] = messages[-1]
    
    return state

In [92]:

def naive_graph():

    # Initialize the graph with the State structure
    graph_builder = StateGraph(State)

    # Add the node to the graph
    graph_builder.add_node('process_prompt', process_prompt)

    # Define the execution flow
    graph_builder.add_edge(START, 'process_prompt')
    graph_builder.add_edge('process_prompt', END)

    # Compile the graph
    graph = graph_builder.compile()

    # Initial state with a prompt
    initial_state = {'prompt': 'Hello, LangGraph!', 'output': ''}

    # Execute the graph
    final_state = graph.invoke(initial_state)

    # Retrieve the output
    print(final_state['output'])  # Outputs: 'HELLO, LANGGRAPH!'
    
naive_graph()

HELLO, LANGGRAPH!


In [99]:
def llm_graph():
    graph_builder = StateGraph(StateUserAI)
    #graph_builder.add_node('process_prompt', process_prompt_llm)
    graph_builder.add_node('process_prompt', process_prompt_user_llm)
    #Define the execution flow
    graph_builder.add_edge(START, 'process_prompt') #Should be the node name from previous step
    graph_builder.add_edge('process_prompt', END)
    
    graph = graph_builder.compile()
    
    #initial_state = {'messages': [HumanMessage(content='I need to fly from New York to San Francisco on the 20th of December. I want to know the cheapest options')]}
    initial_state = {'user_message': [HumanMessage(content='I need to fly from New York to San Francisco on the 20th of December. I want to know the cheapest options')]}
    
    print('Initial State:', initial_state)
    final_state = graph.invoke(initial_state)
    print(final_state)


In [100]:
llm_graph()

Initial State: {'user_message': [HumanMessage(content='I need to fly from New York to San Francisco on the 20th of December. I want to know the cheapest options', additional_kwargs={}, response_metadata={})]}
State: {'user_message': [HumanMessage(content='I need to fly from New York to San Francisco on the 20th of December. I want to know the cheapest options', additional_kwargs={}, response_metadata={})]}
Messages: [SystemMessage(content="You are an assistant to break down a user's query into sub questions. Examine the query and break it down into relevant and appropriate sub questions that can be answered.", additional_kwargs={}, response_metadata={}), HumanMessage(content='I need to fly from New York to San Francisco on the 20th of December. I want to know the cheapest options', additional_kwargs={}, response_metadata={})]
{'user_message': [HumanMessage(content='I need to fly from New York to San Francisco on the 20th of December. I want to know the cheapest options', additional_kwa